# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc0083bd0d0>
├── 'a' --> tensor([[-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941]])
└── 'x' --> <FastTreeValue 0x7fc0083bd520>
    └── 'c' --> tensor([[ 0.5583,  0.0673,  0.7767, -0.0027],
                        [ 0.2225,  0.1238,  0.1468, -0.7128],
                        [ 0.5592, -1.2376, -2.0921, -2.0153]])

In [4]:
t.a

tensor([[-0.8996, -0.9988,  0.4572],
        [ 0.3706, -0.5591, -0.1941]])

In [5]:
%timeit t.a

71.6 ns ± 0.0495 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc0083bd0d0>
├── 'a' --> tensor([[-0.7755,  0.6422,  0.6405],
│                   [-1.1932,  0.2794,  0.0169]])
└── 'x' --> <FastTreeValue 0x7fc0083bd520>
    └── 'c' --> tensor([[ 0.5583,  0.0673,  0.7767, -0.0027],
                        [ 0.2225,  0.1238,  0.1468, -0.7128],
                        [ 0.5592, -1.2376, -2.0921, -2.0153]])

In [7]:
%timeit t.a = new_value

72.2 ns ± 0.122 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941]]),
    x: Batch(
           c: tensor([[ 0.5583,  0.0673,  0.7767, -0.0027],
                      [ 0.2225,  0.1238,  0.1468, -0.7128],
                      [ 0.5592, -1.2376, -2.0921, -2.0153]]),
       ),
)

In [10]:
b.a

tensor([[-0.8996, -0.9988,  0.4572],
        [ 0.3706, -0.5591, -0.1941]])

In [11]:
%timeit b.a

64.8 ns ± 0.0655 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3970, -2.7109,  1.7050],
               [-0.1942, -1.7102, -0.6237]]),
    x: Batch(
           c: tensor([[ 0.5583,  0.0673,  0.7767, -0.0027],
                      [ 0.2225,  0.1238,  0.1468, -0.7128],
                      [ 0.5592, -1.2376, -2.0921, -2.0153]]),
       ),
)

In [13]:
%timeit b.a = new_value

565 ns ± 0.733 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

909 ns ± 1.54 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 57.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

148 µs ± 574 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 839 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbf3c58afd0>
├── 'a' --> tensor([[[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]],
│           
│                   [[-0.8996, -0.9988,  0.4572],
│                    [ 0.3706, -0.5591, -0.1941]]])
└── 'x' --> <FastTreeValue 0x7fbf3c58aca0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.8 µs ± 41.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbf3c5b3f40>
├── 'a' --> tensor([[-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941],
│                   [-0.8996, -0.9988,  0.4572],
│                   [ 0.3706, -0.5591, -0.1941]])
└── 'x' --> <FastTreeValue 0x7fbf3c683370>
    └── 'c' --> tensor([[ 0.5583,  0.0673,  0.7767, -0.0027],
                        [ 0.2225,  0.1238,  0.1468, -0.7128],
                 

In [23]:
%timeit t_cat(trees)

31.8 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.8 µs ± 32.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]],
       
               [[-0.8996, -0.9988,  0.4572],
                [ 0.3706, -0.5591, -0.1941]]]),
    x: Batch(
           c: tensor([[[ 0.5583,  0.0673,  0.7767, -0.0027],
                       [ 0.2225,  0.1238,  0.1468, -0.7128],
                       [ 0.5592, -1.2376, -2.0921, -2.0153]],
         

In [26]:
%timeit Batch.stack(batches)

84.1 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941],
               [-0.8996, -0.9988,  0.4572],
               [ 0.3706, -0.5591, -0.1941]]),
    x: Batch(
           c: tensor([[ 0.5583,  0.0673,  0.7767, -0.0027],
                      [ 0.2225,  0.1238,  0.1468, -0.7128],
                      [ 0.5592, -1.2376, -2.0921, -2.0153],
                      [ 0.5583,  0.0673,  0.7767, -0.0027],
                      [ 0.2225,  

In [28]:
%timeit Batch.cat(batches)

153 µs ± 352 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

354 µs ± 7.02 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
